### HW1 Создание и нормализация базы данных

#### Задачи

* Продумать структуру БД и отрисовать в редакторе;
* Нормализовать БД (1НФ-3НФ), описав, к какой нормальной форме приводится таблица и почему таблица в этой нормальной форме изначально не находилась;
* Создать все таблицы в DBeaver, указав первичные ключи к таблицам, правильные типы данных, могут ли поля быть пустыми или нет;
* Загрузить данные в таблицы в соответствии с созданной структурой.

#### Решение

### Проанализируем исходный датасет

Исходный датасет представлени в виде документа в формате .xlsx, в котором содержится две таблицы: 'transaction' и 'customer', следующего вида:

transaction:

<img src='./static/transaction.png'>

customer:

<img src='./static/customer.png'>

Глядя на талицы можно выделить четыре основные сущности: transaction, customer, product и address (адреса лучше выделить в отдельный справочник так как у одного покупателя может быть несколько адресов доставки)

Разделим таблицы по сущностям с помощью pandas, но сначала выделим признаки для каждой сущности:

* transaction: 

<img src='./static/transaction_table.png'>

* customer:

<img src='./static/customer_table.png'>

* product:

<img src='./static/product_table.png'>

* address:

<img src='./static/address_table.png'>


In [4]:
import pandas as pd
import openpyxl

Загрузим таблицу transaction:

In [6]:
transaction_df = pd.read_excel('./data/HW1_data.xlsx')
transaction_df.head()

FileNotFoundError: [Errno 2] No such file or directory: './data/HW1_data.xlsx'